In [1]:
import pandas as pd
import time

df = pd.read_csv('labeled_and_scored_comments.csv')

Dataset of Wikipedia comments made available by Jigsaw, a subsidiary of Google that created the Perspective tool.

In [2]:
df.sort_values(['score'])

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate
1158,61a85266de70cca0,Можешь говорить по русски.,0.000004,0,0,0,0,0,0
5246,030a9ba0b32b4456,Rodzina Krusenstern Potomkowie Admirała Adama ...,0.000009,0,0,0,0,0,0
11040,a6814193ac6b4705,The opening paragraph states that the game is ...,0.000091,0,0,0,0,0,0
24192,f0d6dee96e057a5b,The book is used as a source in a number of ar...,0.000116,0,0,0,0,0,0
2332,cbb9268d3c7429c1,Vals Vienés \n |Rumba \n |Cha Cha Chá(ciclo 7),0.000131,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
19994,1429a2c930688317,you fucking piece of shit,0.997278,1,1,1,0,1,0
28728,d82e45eb103ad07b,fuck off you stupid aspy asshole,0.997736,1,1,1,0,1,0
25535,e9b92c6c6b01aeef,FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PIE...,0.997982,1,1,1,0,1,0
25945,e783fd267f3a9d3b,FUCK WIKIPEDIA ON WHEELS! \n\nFuck off wikiped...,0.998136,1,1,1,0,1,0


function to make calls to the Perspective API for toxicity score testing

In [3]:
from googleapiclient.discovery import build
import json

def get_toxicity_score(comment):
    
  API_KEY = '' # Put your API key here
    
  client = build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=API_KEY,
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
  static_discovery=False,
  )

  analyze_request = {
  'comment': { 'text': comment },
  'requestedAttributes': {'TOXICITY': {}}
  }
    
  response = client.comments().analyze(body=analyze_request).execute()
  toxicity_score = response["attributeScores"]["TOXICITY"]["summaryScore"]["value"]
    
  return toxicity_score

Get the 50 smallest scores from the csv

In [4]:
df.nsmallest(n=50, columns=['score'])

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate
1158,61a85266de70cca0,Можешь говорить по русски.,0.000004,0,0,0,0,0,0
5246,030a9ba0b32b4456,Rodzina Krusenstern Potomkowie Admirała Adama ...,0.000009,0,0,0,0,0,0
11040,a6814193ac6b4705,The opening paragraph states that the game is ...,0.000091,0,0,0,0,0,0
24192,f0d6dee96e057a5b,The book is used as a source in a number of ar...,0.000116,0,0,0,0,0,0
2332,cbb9268d3c7429c1,Vals Vienés \n |Rumba \n |Cha Cha Chá(ciclo 7),0.000131,0,0,0,0,0,0
29295,d500529f87773731,Pakistan and Irak centrifuges \n\nThere is som...,0.000139,0,0,0,0,0,0
693,379ede734e2fd717,| date2 = 8 April 2015 | result2 = no consensu...,0.000191,0,0,0,0,0,0
1825,9d66cc821b68e7f7,""" \n\n :""""Most scholars find that these argume...",0.000194,0,0,0,0,0,0
18226,2fced21d1a5d0dbd,Tanakh \n\nTanakh is a Hebrew ACRONYM for:\n\n...,0.000205,0,0,0,0,0,0
39533,9e18066512fe6bc6,"In reality Christian Cage DID defend a title, ...",0.000225,0,0,0,0,0,0


Get the 50 largers scores

In [5]:
df.nlargest(n=50, columns=['score'])

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate
17833,364b6c24210e2345,Fuck you you fucking pig!\n\nYou motherfucking...,0.998329,1,0,1,0,1,0
25945,e783fd267f3a9d3b,FUCK WIKIPEDIA ON WHEELS! \n\nFuck off wikiped...,0.998136,1,1,1,0,1,0
25535,e9b92c6c6b01aeef,FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PIE...,0.997982,1,1,1,0,1,0
28728,d82e45eb103ad07b,fuck off you stupid aspy asshole,0.997736,1,1,1,0,1,0
19994,1429a2c930688317,you fucking piece of shit,0.997278,1,1,1,0,1,0
6964,e88321c89a619419,Stupid fucking bitch rot in hell piece of shit,0.996872,1,1,1,0,1,0
35203,b567469c10620312,FUCK YOU FUCK YOU FUCK YOUa,0.996707,1,1,1,0,1,0
18792,26fed2949c2dadc1,you son of a bitch RIP you motherfucker,0.996669,1,0,1,0,1,0
33772,bd4b78896967f1e5,"Hello ScrapIronIV, where is you problem you mo...",0.996289,1,1,1,0,1,0
22535,f9634e62820eed00,"FUCK YOU!!!!=\n FUCK YOU, YOU STUPID BASTARD...",0.996251,1,1,1,0,1,0


Get some basic stats from the score column 

In [6]:
df['score'].describe()

count    41338.000000
mean         0.244467
std          0.257221
min          0.000004
25%          0.074772
50%          0.128969
75%          0.310894
max          0.998329
Name: score, dtype: float64

Create a new column that will add the total tally for each row

In [7]:
df['Total_Score'] = df['toxic'] + df['severe_toxic'] + df['obscene'] + df['threat'] + df['insult'] + df['identity_hate']

Get the first 10 rows to make sure that the the new column 'Total_Score' was created and updated

In [8]:
df.head(10)

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,Total_Score
0,0001ea8717f6de06,Thank you for understanding. I think very high...,0.075638,0,0,0,0,0,0,0
1,000247e83dcc1211,:Dear god this site is horrible.,0.450459,0,0,0,0,0,0,0
2,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0.667964,0,0,0,0,0,0,0
3,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0.068434,0,0,0,0,0,0,0
4,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0.151724,0,0,0,0,0,0,0
5,000663aff0fffc80,this other one from 1897,0.146398,0,0,0,0,0,0,0
6,000689dd34e20979,== Reason for banning throwing == \n\n This ar...,0.091800,0,0,0,0,0,0,0
7,000844b52dee5f3f,|blocked]] from editing Wikipedia. |,0.267695,0,0,0,0,0,0,0
8,00091c35fa9d0465,"== Arabs are committing genocide in Iraq, but ...",0.635929,1,0,0,0,0,0,1
9,000968ce11f5ee34,Please stop. If you continue to vandalize Wiki...,0.604778,0,0,0,0,0,0,0


create a variable that hold all the rows with a Total_Score of 0 

In [9]:
pd = df[df['Total_Score']==0]

In [10]:
pd

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,Total_Score
0,0001ea8717f6de06,Thank you for understanding. I think very high...,0.075638,0,0,0,0,0,0,0
1,000247e83dcc1211,:Dear god this site is horrible.,0.450459,0,0,0,0,0,0,0
2,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0.667964,0,0,0,0,0,0,0
3,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0.068434,0,0,0,0,0,0,0
4,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0.151724,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
41331,9482b95e6ea61cf1,"removing left sidebar \n\nHi,\nYour page is th...",0.082912,0,0,0,0,0,0,0
41334,9480115b93762c35,"Crack, also called cocaine is very good for yo...",0.240977,0,0,0,0,0,0,0
41335,947f206c14a85d9d,REDIRECT Talk:The Test (The O.C.),0.012671,0,0,0,0,0,0,0
41336,947e3f5f2d613605,A question concerning your edit to my user-pag...,0.227474,0,0,0,0,0,0,0


get basic stats for the rows with a score of 0

In [11]:
pd['score'].describe()

count    37160.000000
mean         0.178876
std          0.166702
min          0.000004
25%          0.070867
50%          0.111729
75%          0.234777
max          0.977703
Name: score, dtype: float64

show the rows with the smallest score with a total tally of 0

In [12]:
pd.nsmallest(n=10, columns=['score'])

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,Total_Score
1158,61a85266de70cca0,Можешь говорить по русски.,0.000004,0,0,0,0,0,0,0
5246,030a9ba0b32b4456,Rodzina Krusenstern Potomkowie Admirała Adama ...,0.000009,0,0,0,0,0,0,0
11040,a6814193ac6b4705,The opening paragraph states that the game is ...,0.000091,0,0,0,0,0,0,0
24192,f0d6dee96e057a5b,The book is used as a source in a number of ar...,0.000116,0,0,0,0,0,0,0
2332,cbb9268d3c7429c1,Vals Vienés \n |Rumba \n |Cha Cha Chá(ciclo 7),0.000131,0,0,0,0,0,0,0
29295,d500529f87773731,Pakistan and Irak centrifuges \n\nThere is som...,0.000139,0,0,0,0,0,0,0
693,379ede734e2fd717,| date2 = 8 April 2015 | result2 = no consensu...,0.000191,0,0,0,0,0,0,0
1825,9d66cc821b68e7f7,""" \n\n :""""Most scholars find that these argume...",0.000194,0,0,0,0,0,0,0
18226,2fced21d1a5d0dbd,Tanakh \n\nTanakh is a Hebrew ACRONYM for:\n\n...,0.000205,0,0,0,0,0,0,0
39533,9e18066512fe6bc6,"In reality Christian Cage DID defend a title, ...",0.000225,0,0,0,0,0,0,0


show the rows with the largest score with a total tally of 0

In [13]:
pd.nlargest(n=10, columns=['score'])

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,Total_Score
1057,56902f9d41d0696c,fuck y'all all of yall,0.977703,0,0,0,0,0,0,0
3884,e4c69d7040f12681,"""Whatever everyone knowz """"B"""" is his ONE&ONLY...",0.973927,0,0,0,0,0,0,0
4436,59581cdba9297d11,== YOU suck == \n YOU SUCK ~~~~Insert non-form...,0.967137,0,0,0,0,0,0,0
2426,d72fbec157ade08e,Shout out to https://en.wikipedia.org/wiki/Use...,0.959467,0,0,0,0,0,0,0
3075,9ed499e6a13328e2,jesus is completely fake plz go away fags,0.955174,0,0,0,0,0,0,0
18710,286c1d0f6a3e9e85,Khurki is a dumb Assk Jeeves?,0.949928,0,0,0,0,0,0,0
40576,988d4bc7265812f7,I'm going to go with both but I could be wrong...,0.949928,0,0,0,0,0,0,0
1359,71b808d9d85cd9c0,""" \n\n == Propose rename to """"Pretentious assh...",0.948338,0,0,0,0,0,0,0
520,281a24af9612bca8,mattythewhite is a prick.,0.948086,0,0,0,0,0,0,0
16945,4459bdf71635b1c3,LOL PENIS! \n\nLOL!\n\nI AM TALKING TO YOU SO ...,0.948086,0,0,0,0,0,0,0


create a variable that hold all the rows with a Total_Score of 1 / and get basic stats for row with a total score of 1

In [14]:
pd1 = df[df['Total_Score']==1]
pd1['score'].describe()

count    1595.000000
mean        0.700624
std         0.185652
min         0.054399
25%         0.587656
50%         0.744822
75%         0.846881
max         0.991267
Name: score, dtype: float64

show the rows with the largest score with a total tally of 1

In [15]:
pd1.nlargest(n=10, columns=['score'])

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,Total_Score
32867,c24719adc1207e0c,UR SUCH STUPID U IDIOT \n\nI HATE U I HATE U I...,0.991267,1,0,0,0,0,0,1
17647,39411f0f74479b28,yo bro \n\ni'd understand that you 24 hour ban...,0.989706,1,0,0,0,0,0,1
14808,6844f8c1360f8de0,"I hate you, I am just a kid and you deleted my...",0.987261,1,0,0,0,0,0,1
37718,a7c1979e629c69d7,you will burn in hell \n\nyou are a nazi piece...,0.980125,1,0,0,0,0,0,1
25090,ec1177017948b5f8,It's not vandalism. Are you mentally retarded...,0.979606,1,0,0,0,0,0,1
37558,a89713e29792fed7,Jackass\n\nYou are such a jackass.,0.976422,1,0,0,0,0,0,1
7996,d77936bc6f6122d1,i hate you for deleting my stuff im going to h...,0.974252,1,0,0,0,0,0,1
11117,a4f203360b09a51e,You piece of chickenshit cunt \n\nI'm going to...,0.974174,1,0,0,0,0,0,1
18443,2c7006dd603bf30c,Is it true. \n\nThat you're a mentally ill qua...,0.973020,1,0,0,0,0,0,1
20153,1184b426177b8ff8,"Hello,sorry to waste you're time \n\nhello, so...",0.969740,1,0,0,0,0,0,1


show the rows with the smallest score with a total tally of 1

In [16]:
pd1.nsmallest(n=10, columns=['score'])

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,Total_Score
34556,b8faa0ed6f557ad9,And we have a winner for the douchiest comment...,0.054399,1,0,0,0,0,0,1
30103,d0bae6a8c78773bf,"Look, what's your problem, kid? Have you got s...",0.079995,1,0,0,0,0,0,1
15367,5f04a07ebb59fa3d,Alansohn tucks his sack back every now and the...,0.082855,1,0,0,0,0,0,1
19650,1942efd2dad5e9c5,"You need to end this now, cold turkey.",0.134848,1,0,0,0,0,0,1
38493,a39b4836714430a1,SMOKE WEED ERRYDAY RIGHT BEFORE CLASS....PROFE...,0.140743,1,0,0,0,0,0,1
19334,1e92f8a2a0c261d0,Everyone knows Tony Sidaway is gay.,0.142108,1,0,0,0,0,0,1
14476,6e019e391650cfcc,Honestly the way wiki is treating this is a JO...,0.144263,1,0,0,0,0,0,1
12725,8b0fa7a608ead8ca,"... \n\nHi, I have no idea what you are babbli...",0.150212,1,0,0,0,0,0,1
38101,a5b571ee28eeef5d,they have poor theories,0.153610,1,0,0,0,0,0,1
7931,d8af5b80eed3fcd0,You're cool\n\nYou seem like a really cool guy...,0.154924,1,0,0,0,0,0,1


create a variable that hold all the rows with a Total_Score of 2 / and get basic stats for row with a total score of 2

In [17]:
pd2 = df[df['Total_Score']==2]
pd2['score'].describe()

count    923.000000
mean       0.855190
std        0.110892
min        0.299224
25%        0.806061
50%        0.885550
75%        0.932402
max        0.993684
Name: score, dtype: float64

show the rows with the largest score with a total tally of 2

In [18]:
pd2.nsmallest(n=10, columns=['score'])

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,Total_Score
29490,d403b7f81a7cb054,PIKACHU OPENED UP WIDE AND FILLED WITH SEMEN F...,0.299224,1,0,1,0,0,0,2
11489,9e63db9f1dbbd99a,Hey \n\nAnd you're telling me that without a g...,0.310894,1,0,0,0,1,0,2
34894,b72739ed284e7e2a,Materazzi is Italy's disgrace. The vistory of ...,0.365848,1,0,0,0,1,0,2
12605,8d095ba1caddf5a5,Phil Knight why don't you die?,0.394403,1,0,0,1,0,0,2
4566,3e5e61e3886c413d,PEOPLE LIKE YOU LIKE MEn IN BED WITH OWEN HAHA...,0.396450,1,0,1,0,0,0,2
3176,71f12c0694d0e841,AGENT z and x and tha secretary ROX MY Fking s...,0.403208,1,0,1,0,0,0,2
15494,5d09b188f699e545,@Ganesha \n\nyes now I can understand how many...,0.403282,1,0,0,0,1,0,2
24286,f052b5fb53d79e12,Bad admin-\nBad admin-\nBad admin-\nBad admin-...,0.415776,1,0,0,0,1,0,2
16901,4519a3e23491086e,"""\n\n Dear soldier of the Wikipedia neo-nazi/c...",0.467620,1,0,0,0,1,0,2
21830,fcf25c3563b8c1c4,Lazy layabouts \n\nWant to live without workin...,0.476814,1,0,0,0,1,0,2


show the rows with the largest score with a total tally of 2

In [19]:
pd2.nlargest(n=10, columns=['score'])

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,Total_Score
17349,3e1309cf82d7bdff,you are dumb \n\nyou are dumb and stupid loser,0.993684,1,0,0,0,1,0,2
35766,b239a301554a2ed0,Fuck You! \n\nfuck you. fuck you. fuck you. fu...,0.992689,1,0,1,0,0,0,2
19674,18f35bf40d79d277,FUCK YOU!!!! stupid IGNORANT!!!!! \n\nHIM\nis ...,0.991811,1,0,1,0,0,0,2
61,03c45b72d38c1f75,FUCK OFF FUCK OFF FUCK OFF FUCK OFF FUCK OFF F...,0.989706,1,0,1,0,0,0,2
2717,fb03b345b59a2875,by the way i fucked ur mum in the arse hole,0.989706,1,0,1,0,0,0,2
10293,b29752d82fe0beec,Mangojuice is a piece of shit that should fuck...,0.989706,1,0,1,0,0,0,2
16381,4f6e970e3995a98d,antares\n\nthis is one big motherfucker,0.989706,1,0,1,0,0,0,2
24236,f0960e1fadfc88a5,"Personal attacks (from Annonymous proxy)\nHi, ...",0.989706,1,0,0,0,1,0,2
34654,b8747a1278d44ee3,I will kill you \n\ni will blow up your house ...,0.989706,1,0,0,1,0,0,2
34978,b6a190ce27684886,WHAT KIND OF NAME IS SUNSHINE YOU ARE AN IDIOT,0.989706,1,0,0,0,1,0,2


create a variable that hold all the rows with a Total_Score of 3 / and get basic stats for row with a total score of 3

In [20]:
pd3 = df[df['Total_Score']==3]
pd3['score'].describe()

count    1107.000000
mean        0.923507
std         0.080341
min         0.307727
25%         0.901717
50%         0.949330
75%         0.974761
max         0.998329
Name: score, dtype: float64

show the rows with the smallest score with a total tally of 3

In [21]:
pd1.nsmallest(n=10, columns=['score'])

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,Total_Score
34556,b8faa0ed6f557ad9,And we have a winner for the douchiest comment...,0.054399,1,0,0,0,0,0,1
30103,d0bae6a8c78773bf,"Look, what's your problem, kid? Have you got s...",0.079995,1,0,0,0,0,0,1
15367,5f04a07ebb59fa3d,Alansohn tucks his sack back every now and the...,0.082855,1,0,0,0,0,0,1
19650,1942efd2dad5e9c5,"You need to end this now, cold turkey.",0.134848,1,0,0,0,0,0,1
38493,a39b4836714430a1,SMOKE WEED ERRYDAY RIGHT BEFORE CLASS....PROFE...,0.140743,1,0,0,0,0,0,1
19334,1e92f8a2a0c261d0,Everyone knows Tony Sidaway is gay.,0.142108,1,0,0,0,0,0,1
14476,6e019e391650cfcc,Honestly the way wiki is treating this is a JO...,0.144263,1,0,0,0,0,0,1
12725,8b0fa7a608ead8ca,"... \n\nHi, I have no idea what you are babbli...",0.150212,1,0,0,0,0,0,1
38101,a5b571ee28eeef5d,they have poor theories,0.153610,1,0,0,0,0,0,1
7931,d8af5b80eed3fcd0,You're cool\n\nYou seem like a really cool guy...,0.154924,1,0,0,0,0,0,1


show the row with the largest score with a total tally of 3

In [22]:
pd3.nlargest(n=10, columns=['score'])

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,Total_Score
17833,364b6c24210e2345,Fuck you you fucking pig!\n\nYou motherfucking...,0.998329,1,0,1,0,1,0,3
18792,26fed2949c2dadc1,you son of a bitch RIP you motherfucker,0.996669,1,0,1,0,1,0,3
25733,e89e826751cffb30,"Go fuck yourself, you are shit human. 220.253....",0.995935,1,0,1,0,1,0,3
23096,f68d9ef333909f03,Fucking whore \n\nYou are such a dumb bitch i ...,0.995611,1,0,1,0,1,0,3
21578,fe583e3374e6df30,FUCK YOU \n\nFUCK YOU AGAIN,0.995440,1,0,1,0,1,0,3
1695,918b665e3026cbef,FUCK YOU ALL! FUCK YOU ALL! FUCK YOU ALL! FUCK...,0.994959,1,0,1,0,1,0,3
38412,a41007da2c8bad58,"fuck you, idiot. you don't even know the subje...",0.994802,1,0,1,0,1,0,3
38175,a55806a1069a394d,You suck! \n\nI hate your guts you piece of sh...,0.994799,1,0,1,0,1,0,3
35325,b4c25c7cc7d728f5,ugly fucker who sucks cocks,0.994715,1,0,1,0,1,0,3
27643,de36b31e15959013,"""\n\n Suck my cunt \n\nyou bastard!""""""",0.994657,1,0,1,0,1,0,3


create a variable that hold all the rows with a Total_Score of 4 / and get basic stats for row with a total score of 4

In [23]:
pd4 = df[df['Total_Score']==4]
pd4['score'].describe()

count    434.000000
mean       0.955829
std        0.048769
min        0.680817
25%        0.948086
50%        0.973544
75%        0.989706
max        0.998136
Name: score, dtype: float64

show the rows with the smallest score with a total tally of 4

In [24]:
pd4.nsmallest(n=10, columns=['score'])

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,Total_Score
33118,c0e076f0011acaa6,"""FFFFF UUUUUU CCCCCC KKKKKK ===== YOU! F UU C ...",0.680817,1,1,1,0,1,0,4
2479,de7c589398ba3ee2,"Now e'rybody, e'rybody, e'rybody, e'rybody \n ...",0.744838,1,0,1,0,1,1,4
34115,bb5a10f7644658cf,"you are a prat, and my revenge will be sweet! ...",0.744987,1,0,1,1,1,0,4
10146,b56bd64c1b28345f,"""\n\nhehe, you *ARE* truly pathetic Zereshk, a...",0.749619,1,0,1,0,1,1,4
25371,eab35a94edfe8946,Remeber rahmatul lil alameen? You Sunni monkey...,0.768935,1,1,0,0,1,1,4
2492,de9ccf9b087476fb,*Eggot Filipino word for 'Nigger'. Originated ...,0.770228,1,0,1,0,1,1,4
9263,c35fa79b4feb3d92,"And another thing, stay off Macedonians (ethni...",0.783836,1,0,1,0,1,1,4
35479,b3d931f3b33b57fa,"""A RESPONSE TO . I ENTITLE THIS """"YOUR STUPIDI...",0.785457,1,0,1,1,1,0,4
30556,ce26cedc315e5d96,"Objectivism \n\nHey Walter, dont you remember ...",0.785726,1,0,1,0,1,1,4
21698,fdcb97eb7bdf1707,DAT CHICK \n\nOMG I DIDNT VANDALISE NUFFINK YH...,0.794439,1,1,1,0,1,0,4


show the rows with the largest score with a total tally of 4

In [25]:
pd4.nlargest(n=10, columns=['score'])

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,Total_Score
25945,e783fd267f3a9d3b,FUCK WIKIPEDIA ON WHEELS! \n\nFuck off wikiped...,0.998136,1,1,1,0,1,0,4
25535,e9b92c6c6b01aeef,FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PIE...,0.997982,1,1,1,0,1,0,4
28728,d82e45eb103ad07b,fuck off you stupid aspy asshole,0.997736,1,1,1,0,1,0,4
19994,1429a2c930688317,you fucking piece of shit,0.997278,1,1,1,0,1,0,4
6964,e88321c89a619419,Stupid fucking bitch rot in hell piece of shit,0.996872,1,1,1,0,1,0,4
35203,b567469c10620312,FUCK YOU FUCK YOU FUCK YOUa,0.996707,1,1,1,0,1,0,4
33772,bd4b78896967f1e5,"Hello ScrapIronIV, where is you problem you mo...",0.996289,1,1,1,0,1,0,4
22535,f9634e62820eed00,"FUCK YOU!!!!=\n FUCK YOU, YOU STUPID BASTARD...",0.996251,1,1,1,0,1,0,4
18034,32d855a249fd09e0,fuck you \n\nFUCK YOU MOTHERFUCKER!!!! GO DIE ...,0.996052,1,1,1,0,1,0,4
35749,b25c87ee437e7e51,why? \n\nfuck you you fucking whore.\nyou stup...,0.996048,1,1,1,0,1,0,4


create a variable that hold all the rows with a Total_Score of 5 / and get basic stats for row with a total score of 5

In [26]:
pd5 = df[df['Total_Score']==5]
pd5['score'].describe()

count    111.000000
mean       0.964023
std        0.038040
min        0.715893
25%        0.958952
50%        0.975392
75%        0.986112
max        0.996221
Name: score, dtype: float64

show the rows with the largest score with a total tally of 5

In [27]:
pd5.nsmallest(n=10, columns=['score'])

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,Total_Score
28910,d73516d5de7f7384,big>I HATE Nl33ERS\nI HATE Nl33ERS\nI HATE Nl3...,0.715893,1,1,1,0,1,1,5
19145,21a8d6268f06bd50,YOu Are A SOn of Bitch Turk\n\nJust to let you...,0.798335,1,1,1,0,1,1,5
9194,c47353648be65580,BUSH NUKE IRAN!!! IT IS ARABIAN GULF STUPID PE...,0.870418,1,0,1,1,1,1,5
40423,9973cc45a10db3f5,"""\n\nhaha you are a dumb shit. what's so wrong...",0.892599,1,1,1,0,1,1,5
20064,12f877656b9f82c9,"You Dick \n\nYou're a big, anti-Japanese dick,...",0.896489,1,1,1,0,1,1,5
26939,e21d4a0f9fc3f335,Hey u crowbeat ... Kshatriyas rule the world....,0.901734,1,1,1,0,1,1,5
25379,eaa76f9c074e3aa5,Daedalus is nothing but a filthy n!gger and a ...,0.901749,1,1,1,0,1,1,5
2558,e5fa9d0d593b7d06,"== WHAT is going on here??? == \n\n Aug 23, 10...",0.903912,1,1,1,0,1,1,5
23702,f36374af17471824,THE FUKKING RETARDO RACISTO KANUCK,0.907424,1,1,1,0,1,1,5
16449,4e875d3dcff0d329,"""u are a fucking wanker!!!! There was no fucke...",0.919593,1,1,1,0,1,1,5


show the rows with the largest score with a total tally of 5

In [28]:
pd5.nlargest(n=8, columns=['score'])

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,Total_Score
27863,dcfe7014ec173fa8,Fuck you Indian piece of shit.,0.996221,1,1,1,0,1,1,5
31742,c81e3c4002cad452,== fuck you \nI hope you die motherfucker,0.995264,1,1,1,1,1,0,5
21407,ff3c218c3be4d2d4,fucking germanian piece of shit,0.994861,1,1,1,0,1,1,5
16436,4ebb0ac23020ce07,your gay \n\nyou're a fucking faggot,0.994043,1,1,1,0,1,1,5
332,1a44cad92a0c1c96,==Son of a bitch== \n\n Hey you fucking neo-Na...,0.992705,1,1,1,0,1,1,5
26540,e42607741bd5800c,Oh of fuck you asshole die im going to kill yo...,0.992081,1,1,1,1,1,0,5
30716,cd631564b47beac5,==\n\nFuck you. Fuck everyone on this site. Yo...,0.991621,1,1,1,0,1,1,5
20488,0c5a912e6830c660,You piece of shit fucking block me then pussy....,0.991315,1,1,1,0,1,1,5


create a variable that hold all the rows with a Total_Score of 6 / and get basic stats for row with a total score of 6

In [29]:
pd6 = df[df['Total_Score']==6]
pd6['score'].describe()

count    8.000000
mean     0.967222
std      0.012812
min      0.949070
25%      0.955972
50%      0.971108
75%      0.977773
max      0.980357
Name: score, dtype: float64

show the rows with the smalles score with a total tally of 6

In [30]:
pd1.nsmallest(n=10, columns=['score'])

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,Total_Score
34556,b8faa0ed6f557ad9,And we have a winner for the douchiest comment...,0.054399,1,0,0,0,0,0,1
30103,d0bae6a8c78773bf,"Look, what's your problem, kid? Have you got s...",0.079995,1,0,0,0,0,0,1
15367,5f04a07ebb59fa3d,Alansohn tucks his sack back every now and the...,0.082855,1,0,0,0,0,0,1
19650,1942efd2dad5e9c5,"You need to end this now, cold turkey.",0.134848,1,0,0,0,0,0,1
38493,a39b4836714430a1,SMOKE WEED ERRYDAY RIGHT BEFORE CLASS....PROFE...,0.140743,1,0,0,0,0,0,1
19334,1e92f8a2a0c261d0,Everyone knows Tony Sidaway is gay.,0.142108,1,0,0,0,0,0,1
14476,6e019e391650cfcc,Honestly the way wiki is treating this is a JO...,0.144263,1,0,0,0,0,0,1
12725,8b0fa7a608ead8ca,"... \n\nHi, I have no idea what you are babbli...",0.150212,1,0,0,0,0,0,1
38101,a5b571ee28eeef5d,they have poor theories,0.153610,1,0,0,0,0,0,1
7931,d8af5b80eed3fcd0,You're cool\n\nYou seem like a really cool guy...,0.154924,1,0,0,0,0,0,1


show the rows with the largest score with a total tally of 6

In [31]:
pd6.nlargest(n=8, columns=['score'])

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,Total_Score
22370,fa3c0dc48563a1f8,faggot \n\nYou lil piece of shit I haven’t van...,0.980357,1,1,1,1,1,1,6
1449,799638b93f93a0c4,::Damn you cocksuckers! FUCK YOU AND YOUR MOM....,0.980008,1,1,1,1,1,1,6
13887,77d84b1321c22d9a,"LGBT \n\nyou little fuck , are you a fag , tha...",0.977029,1,1,1,1,1,1,6
7613,ddf6dc5cf6931f48,Ok.... \n\nBitch i swear to God i will fuckin ...,0.974249,1,1,1,1,1,1,6
41017,9622c82f97eb26cd,She is a Motherfucker \n\nI will block you Mot...,0.967966,1,1,1,1,1,1,6
31307,ca7c493a2a81b65f,"and your little faggot boy Propol, pray, pray ...",0.957396,1,1,1,1,1,1,6
338,1a4c6302cea3ffe2,Dick Cheney will fucking kill all of those god...,0.951699,1,1,1,1,1,1,6
4312,c26941cbaad63533,"""Oi, dickhead- you call Straight Edge """"Gay"""" ...",0.949070,1,1,1,1,1,1,6


Create comment list, loop throught get_toxicity_score function and get comment and score

In [32]:
comment_list = ['stupid', 'how are you', 'fine thanks']

for comment in comment_list:
    score = get_toxicity_score(comment)
    print(comment, score, )
    time.sleep(1)

stupid 0.9147037
how are you 0.073404066
fine thanks 0.03189806


create a list of namjoon hate comments

In [33]:
hate_namjoon_comments = ['B*tch i told you all started it now stfu and worry about namjoon\'s ugly face', "How many times u keep using this edited pict? Here\'s the non edited pict of ugly namjoon", "Namjoon and Jhope are ugly jimin taehyung and jin can\’t sing and jin a bitch running from enlistment FUCK BTS", 
 'what’s botched ass hwasa doing? what’s botched ass namjoon doing? worry but the people you stan cause they ugly in the face and ain’t doing shit', 'namjoon ugly ass deserve to be enlisted', 'One ugly bitvh just called namjoon ugly like tfff... I wanna b3at her a5s']

get a toxicity score for the hate namjoon comments 

In [34]:
for comment in hate_namjoon_comments:
    score = get_toxicity_score(comment)
    print(comment, score )
    time.sleep(1)

B*tch i told you all started it now stfu and worry about namjoon's ugly face 0.9063065
How many times u keep using this edited pict? Here's the non edited pict of ugly namjoon 0.794439
Namjoon and Jhope are ugly jimin taehyung and jin can\’t sing and jin a bitch running from enlistment FUCK BTS 0.9592989
what’s botched ass hwasa doing? what’s botched ass namjoon doing? worry but the people you stan cause they ugly in the face and ain’t doing shit 0.94992846
namjoon ugly ass deserve to be enlisted 0.94992846
One ugly bitvh just called namjoon ugly like tfff... I wanna b3at her a5s 0.8274126


Set toxicity levels to categorize the comment scores for namjoon hate comments

In [41]:
max_toxic_threshold = 0.84
min_toxic_threshold = 0.000004
mid_toxic_threshold = 0.12

for comment in hate_namjoon_comments:
    score = get_toxicity_score(comment)
    if score >= min_toxic_threshold and score < mid_toxic_threshold:
        toxic_level = 'Low'
    elif score >= mid_toxic_threshold and score < max_toxic_threshold:
        toxic_level = 'Mid'
    elif score >= max_toxic_threshold:
        toxic_level = 'Max'
    print(comment,':', score, ':', toxic_level )
    time.sleep(1)

B*tch i told you all started it now stfu and worry about namjoon's ugly face : 0.9063065 : Max
How many times u keep using this edited pict? Here's the non edited pict of ugly namjoon : 0.794439 : Mid
Namjoon and Jhope are ugly jimin taehyung and jin can\’t sing and jin a bitch running from enlistment FUCK BTS : 0.9592989 : Max
what’s botched ass hwasa doing? what’s botched ass namjoon doing? worry but the people you stan cause they ugly in the face and ain’t doing shit : 0.94992846 : Max
namjoon ugly ass deserve to be enlisted : 0.94992846 : Max
One ugly bitvh just called namjoon ugly like tfff... I wanna b3at her a5s : 0.8274126 : Mid


create a list of namjoon nice comments

In [36]:
good_namjoon_comments = ['kim namjoon being perfect', 'kim namjoon, most beautiful.', 'My beautiful sexy man I love you This man is so beautiful and sexy to me and I love him', "Namjoon's lips, mouth insists on being kissed! Believe me! And I will say that the boys with beautiful mouths, do not know how to kiss!", 'quickly realizing that i love seeing namjoon eat well', 'Namjoon\'s dimples on his cheeks is the most precious thing']

Get toxicity scores for the good namjoon comments

In [37]:
for comment in good_namjoon_comments:
    score = get_toxicity_score(comment)
    print(comment, score )
    time.sleep(1)

kim namjoon being perfect 0.17438614
kim namjoon, most beautiful. 0.05818944
My beautiful sexy man I love you This man is so beautiful and sexy to me and I love him 0.53490317
Namjoon's lips, mouth insists on being kissed! Believe me! And I will say that the boys with beautiful mouths, do not know how to kiss! 0.4938513
quickly realizing that i love seeing namjoon eat well 0.15049207
Namjoon's dimples on his cheeks is the most precious thing 0.34874594


Set toxicity levels to categorize the comment scores for namjoon hate comments

In [39]:
max_toxic_threshold = 0.84
min_toxic_threshold = 0.000004
mid_toxic_threshold = 0.12

for comment in good_namjoon_comments:
    score = get_toxicity_score(comment)
    if score >= min_toxic_threshold and score < mid_toxic_threshold:
        toxic_level = 'Low'
    elif score >= mid_toxic_threshold and score < max_toxic_threshold:
        toxic_level = 'Mid'
    elif score >= max_toxic_threshold:
        toxic_level = 'Max'
    print(comment,':', score, ':', toxic_level )
    time.sleep(1)

kim namjoon being perfect : 0.17438607 : Mid
kim namjoon, most beautiful. : 0.05818944 : Low
My beautiful sexy man I love you This man is so beautiful and sexy to me and I love him : 0.53490317 : Mid
Namjoon's lips, mouth insists on being kissed! Believe me! And I will say that the boys with beautiful mouths, do not know how to kiss! : 0.49385133 : Mid
quickly realizing that i love seeing namjoon eat well : 0.15049209 : Mid
Namjoon's dimples on his cheeks is the most precious thing : 0.34874597 : Mid
